# Hyperparamter Optimization

### GaussianProcessRegressor vs GridSearch

In [1]:
import pandas as pd
from sklearn import svm
from scipy.sparse import csr_matrix
import numpy as np
from sklearn import metrics

from sklearn.gaussian_process import GaussianProcessRegressor
import sklearn.gaussian_process.kernels as kernels

## Arguments

In [2]:
'''
Full path of the file
'''
FILEPATH_TRAIN = 'train.csv'
FILEPATH_TEST = 'test.csv'
'''
The number of classes
'''
NUM_CLASSES = 3

## Util functions

In [3]:
def read_train_test_data():
    df = pd.read_csv(FILEPATH_TRAIN)
    dfTest = pd.read_csv(FILEPATH_TEST)
    return df,dfTest

def extract_train_test_matrixes(df,dfTest):
    X = df.iloc[:,0:len(df.columns)-(NUM_CLASSES+1)].as_matrix()
    y = df.iloc[:,-1].as_matrix()

    Xval = dfTest.iloc[:,0:len(df.columns)-(NUM_CLASSES+1)].as_matrix()
    yval = dfTest.iloc[:,-1].as_matrix()
    
    return X,y,Xval,yval
    

def print_evaluation_summary(labels,predictions):
    
    labels = np.array(labels)
    predictions = np.array(predictions)
    
    correct = np.sum(np.equal(labels,predictions))
    total = len(labels)
    acc = correct/total
    
    print('Dataset information:')
    print('Num observations: %d' % int(labels.shape[0]))
    print('Num classes: %d\n' % NUM_CLASSES)
    
    print('Evaluation:')
    print('Total hits: %d/%d - pct: %.3f\n' % (correct,total,acc))
    
    print('Confusion Matrix:\n')
    print(str(metrics.confusion_matrix(labels,predictions))+'\n')
    
    print('Classification Report: '+'\n')
    print(metrics.classification_report(labels, predictions))

## Read data

In [4]:
df,dfTest = read_train_test_data()

In [5]:
X,y,Xval,yval = extract_train_test_matrixes(df,dfTest)

In [6]:
X.shape,y.shape,Xval.shape,yval.shape

((11013, 18), (11013,), (380, 18), (380,))

## Bayesian Optimization

In [7]:
kernel = kernels.RBF(length_scale_bounds="fixed")
gp = GaussianProcessRegressor(kernel=kernel, random_state=0)
gp.fit(X, y)

GaussianProcessRegressor(alpha=1e-10, copy_X_train=True,
             kernel=RBF(length_scale=1), n_restarts_optimizer=0,
             normalize_y=False, optimizer='fmin_l_bfgs_b', random_state=0)

In [8]:
predGP = np.clip(np.around(a=gp.predict(Xval),decimals=0),a_min=0,a_max=2)

## Grid search

A grid search (done on another Notebook) gives us the next result as best hyperparameters

In [9]:
C = 30
gamma = 0.01

In [10]:
svc = svm.SVC(C=C, gamma=gamma,kernel='rbf')
fitted_svc = svc.fit(X, y)

In [11]:
predGridSearch = svc.predict(Xval)

## Evaluation - Head2Head

In [12]:
print_evaluation_summary(yval,predGridSearch)

Dataset information:
Num observations: 380
Num classes: 3

Evaluation:
Total hits: 203/380 - pct: 0.534

Confusion Matrix:

[[177   0   4]
 [ 83   0   6]
 [ 84   0  26]]

Classification Report: 

             precision    recall  f1-score   support

          0       0.51      0.98      0.67       181
          1       0.00      0.00      0.00        89
          2       0.72      0.24      0.36       110

avg / total       0.45      0.53      0.42       380



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
print_evaluation_summary(yval,predGP)

Dataset information:
Num observations: 380
Num classes: 3

Evaluation:
Total hits: 133/380 - pct: 0.350

Confusion Matrix:

[[79 77 25]
 [43 36 10]
 [30 62 18]]

Classification Report: 

             precision    recall  f1-score   support

          0       0.52      0.44      0.47       181
          1       0.21      0.40      0.27        89
          2       0.34      0.16      0.22       110

avg / total       0.39      0.35      0.35       380

